In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import json_lines
import preprocess_temp as P
import model.parsers as M

### Load Data

In [3]:
directory = './conala-corpus/'
train_file = directory + 'train.json'
test_file = directory + 'test.json'

with open(train_file) as f:
    train_data = json.load(f)
    
with open(test_file) as f:
    test_data = json.load(f)

In [4]:
mine_file = directory + 'mined.jsonl'
mine_data = []
with open(mine_file) as f:
    mine_data = [line for line in json_lines.reader(f)]

### Let's preprocess the data. Everything is in Preprocess.py
### Adding mined data

In [5]:
mine_data[0]

{'id': '34705205_34705233_0',
 'intent': 'Sort a nested list by two elements',
 'parent_answer_post_id': 34705233,
 'prob': 0.8690001442846342,
 'question_id': 34705205,
 'snippet': 'sorted(l, key=lambda x: (-int(x[1]), x[0]))'}

In [6]:
# intent processing includes lowercase, remove punctuation'?'
train_intent, train_codes = P.process_data(train_data)
test_intent, test_codes = P.process_data(test_data)

In [7]:
mine_intent, mine_codes = P.process_data(mine_data, mine=True)

In [8]:
# this class is used for code2actions and actions2code
ast_action = P.Ast_Action()

In [9]:
train_actions = []

for code in train_codes:
    train_actions.append(ast_action.code2actions(code))

In [10]:
word_lst = P.vocab_list(train_intent, cut_freq=2)
act_lst, token_lst = P.action_list(train_actions, cut_freq=0)

In [11]:
word2num = dict(zip(word_lst, range(0,len(word_lst))))
act2num = dict(zip(act_lst, range(0,len(act_lst))))
token2num = dict(zip(token_lst, range(0,len(token_lst))))

In [12]:
train_loader = P.get_train_loader(train_intent, train_actions, word2num, act2num, token2num, batch_size=48)

In [13]:
test_loader = P.get_test_loader(test_intent, word2num, batch_size=1)

In [14]:
action_index_copy = act2num[P.GenTokenAction('copy')]
action_index_gen = act2num[P.GenTokenAction('token')]

### Model

In [15]:
import torch
import time

In [53]:
from collections import namedtuple
hyperParamMap = {
    #### General configuration ####
    'cuda': True,      # Use gpu
    'mode': 'train',   # train or test

    #### Embedding sizes ####
    'embed_size': 128,         # Size of word embeddings
    'action_embed_size': 128,  # Size of ApplyRule/GenToken action embeddings
    'field_embed_size': 64,    # Embedding size of ASDL fields
    'type_embed_size': 64,     # Embeddings ASDL types

    #### Decoding sizes ####
    'hidden_size': 256,        # Size of LSTM hidden states
    'beam_size': 5,
    
    #### training schedule details ####
    'valid_metric': 'acc',                # Metric used for validation
    'valid_every_epoch': 1,               # Perform validation every x epoch
    'log_every': 10,                      # Log training statistics every n iterations
    'save_to': 'model',                   # Save trained model to
    'clip_grad': 5.,                      # Clip gradients
    'max_epoch': 10,                      # Maximum number of training epoches
    'optimizer': 'Adam',                  # optimizer
    'lr': 0.001,                          # Learning rate
    'lr_decay': 0.5,                      # decay learning rate if the validation performance drops
    'verbose': False,                     # Verbose mode

    #### decoding/validation/testing ####
    'load_model': None,                   # Load a pre-trained model
    'beam_size': 1,                       # Beam size for beam search
    'decode_max_time_step': 100,          # Maximum number of time steps used in decoding and sampling
    'sample_size': 5,                     # Sample size
    'test_file': '',                      # Path to the test file
    'save_decode_to': None,               # Save decoding results to file
}

HyperParams = namedtuple('HyperParams', list(hyperParamMap.keys()), verbose=False)
hyperParams = HyperParams(**hyperParamMap)

In [17]:
model = M.Model(hyperParams, action_size=len(act_lst), token_size=len(token_lst), word_size=len(word_lst), 
                      action_index_copy=action_index_copy, action_index_gen=action_index_gen)

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
lossFunc = torch.nn.CrossEntropyLoss()

In [56]:
model.train()
epoch_begin = time.time()
for e in range(20):
    for batch_ind, x in enumerate(train_loader):
        optimizer.zero_grad()

        (action_logits, action_labels), (copy_logits, copy_labels), (token_logits, token_labels) = model(x)

        loss1 = lossFunc(action_logits, action_labels)
        loss2 = torch.DoubleTensor([0.0])
        if len(copy_logits) > 0:
            loss2 = lossFunc(copy_logits, copy_labels)
        loss3 = torch.DoubleTensor([0.0])
        if len(token_logits) > 0:
            loss3 = lossFunc(token_logits, token_labels)

        total_loss = loss1 + loss2.double() + loss3.double()
        total_loss.backward()

        # clip gradient
        if hyperParams.clip_grad > 0.:
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), hyperParams.clip_grad)

        optimizer.step()

        if batch_ind % hyperParams.log_every == hyperParams.log_every - 1:
            print('--------------------------epoch {} batch {}-----------------------------'.format(e, batch_ind))
            print("Action loss: {}".format(loss1.data))
            print("Copy loss: {}".format(loss2.data))
            print("Token loss: {}".format(loss3.data))
            report_loss = report_examples = 0.

    print('epoch elapsed %ds' % (time.time() - epoch_begin))

--------------------------epoch 0 batch 9-----------------------------
Action loss: 0.31797289430155523
Copy loss: 1.6053892374038696
Token loss: 0.5260612964630127


--------------------------epoch 0 batch 19-----------------------------
Action loss: 0.32988686316754373
Copy loss: 1.663701057434082
Token loss: 0.8086149096488953


--------------------------epoch 0 batch 29-----------------------------
Action loss: 0.31746549149973635
Copy loss: 1.4932278394699097
Token loss: 0.703612208366394


--------------------------epoch 0 batch 39-----------------------------
Action loss: 0.42413590459650874
Copy loss: 1.7143309116363525
Token loss: 0.7998818159103394


--------------------------epoch 0 batch 49-----------------------------
Action loss: 0.522713569952576
Copy loss: 1.6760286092758179
Token loss: 0.866177499294281
epoch elapsed 115s


--------------------------epoch 1 batch 9-----------------------------
Action loss: 0.41401701281599795
Copy loss: 1.8249558210372925
Token loss: 0.633781373500824


In [44]:
torch.save((model).state_dict(), 'Parameters/cut_2_0_v4.t7')

In [29]:
model.load_state_dict(torch.load('Parameters/cut_2_0_v2.t7'))

In [45]:
sample_sent, sample_sent_txt = next(iter(test_loader))
print(sample_sent)
print(sample_sent_txt)
sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action, decode_method='greedy')
ast_action.actions2code(sample_hypothesis.actions)

[tensor([1291,    3, 1411, 1411,    8,   19,  252, 1070])]
[['send', 'a', 'signal', '`signal.SIGUSR1`', 'to', 'the', 'current', 'process']]


"os.system('taskkill /im')"

In [161]:
print(sample_hypothesis.actions)
ast_action.actions2code(sample_hypothesis.actions)

[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[any], ApplyRule[expr -> GeneratorExp(expr elt, comprehension* generators)], ApplyRule[expr -> Compare(expr left, cmpop* ops, expr* comparators)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[isinstance], ApplyRule[expr -> Name(identifier id)], GenToken[x], Reduce, Reduce, Reduce, Reduce, ApplyRule[comprehension -> comprehension(expr target, expr iter, expr* ifs)], ApplyRule[expr -> Name(identifier id)], GenToken[i], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[range], ApplyRule[expr -> Num(object n)], GenToken[100], ApplyRule[expr -> Num(object n)], GenToken[9], Reduce, Reduce, Reduce, Reduce, Reduce, Reduce]
Expr(value=Call(func=Name(id='any'), args=[GeneratorExp(elt=Compare(left=Call(func=Name

IndexError: list index out of range

In [51]:
import json

ground_truth = []
for e in test_data:
    ground_truth.append(e['snippet'])

# model.eval()
cnt = 1
code_list = []
bleu_scores = []
import time
import dataset.util as U
import dataset.bleu_score as my_bleu
# import nltk.translate as bleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

t0 = time.time()
test_loader = P.get_test_loader(test_intent, word2num, batch_size=1)
for ind, (sample_sent, sample_sent_txt) in enumerate(test_loader):
    t1 = time.time()
    sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action,
                                    decode_method='greedy')
    try:
        code = ast_action.actions2code(sample_hypothesis.actions)
    except:
        code = '.'
    code_list.append(code)
    t2 = time.time()
    print('----------------------answer {}, this one cost {}s, total cost {}s'.format(cnt, t2 - t1, t2 - t0))
    print('my code:   ', code)
    print('ground truth:   ', ground_truth[ind])
    code_token_list = U.tokenize_for_bleu_eval(code)
    target_token_list = U.tokenize_for_bleu_eval(ground_truth[ind])

    # First argument should be list of list of words from ground truth (in our case only one ground truth)
    # Second argument should be our prediction, and it's a list of word
    bleu = sentence_bleu([target_token_list], code_token_list)
    # bleu, _, _, _, _, _ = my_bleu.compute_bleu([target_token_list], code_token_list)
    bleu_scores.append(bleu)
    print('bleu socre {}'.format(bleu))
    cnt = cnt + 1
print(code_list)
with open('./answer.txt', 'w') as outfile:
    json.dump(code_list, outfile, indent=1)


----------------------answer 1, this one cost 0.03453993797302246s, total cost 0.037541866302490234s
my code:    os.system('taskkill /im')
ground truth:    os.kill(os.getpid(), signal.SIGUSR1)
bleu socre 0.3077772945160715
----------------------answer 2, this one cost 0.028843164443969727s, total cost 0.0672609806060791s
my code:    webbrowser.write('log', 'libs')
ground truth:    bytes.fromhex('4a4b4c').decode('utf-8')
bleu socre 0.37621186824601177
----------------------answer 3, this one cost 0.023617982864379883s, total cost 0.09195184707641602s
my code:    map(i)
ground truth:    all(x == myList[0] for x in myList)
bleu socre 0.04186576731652839
----------------------answer 4, this one cost 0.04301714897155762s, total cost 0.1358928680419922s
my code:    re.findall('filename.png', '"\\1"', 'title')
ground truth:    print('%*s : %*s' % (20, 'Python', 20, 'Very Good'))
bleu socre 0.4569605577587919
----------------------answer 5, this one cost 0.03316617012023926s, total cost 0.1705

/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


----------------------answer 6, this one cost 0.09862995147705078s, total cost 0.27000880241394043s
my code:    pd.DataFrame(key=lambda k=d2.keys(): 'y3')
ground truth:    res = {k: v for k, v in list(kwargs.items()) if v is not None}
bleu socre 0.2966558910876616
----------------------answer 7, this one cost 0.08110284805297852s, total cost 0.3523988723754883s
my code:    pd.DataFrame(key=lambda k=d2.keys(): 'y3')
ground truth:    res = dict((k, v) for k, v in kwargs.items() if v is not None)
bleu socre 0.2869161472089115


----------------------answer 8, this one cost 0.38297605514526367s, total cost 0.7368137836456299s
my code:    os.system(None)
ground truth:    subprocess.check_output('ps -ef | grep something | wc -l', shell=True)
bleu socre 0.06902498108894259
----------------------answer 9, this one cost 0.03671884536743164s, total cost 0.7741677761077881s
my code:    concatenate(concatenate(x, x))
ground truth:    """""".join(['a', 'b', 'c'])
bleu socre 0.1603801045486562
----------------------answer 10, this one cost 0.033818721771240234s, total cost 0.8087806701660156s
my code:    df.loc('type').diff('state')
ground truth:    pd.Series(list(set(s1).intersection(set(s2))))
bleu socre 0.2945901093386716
----------------------answer 11, this one cost 0.018815994262695312s, total cost 0.8284358978271484s
my code:    ax.set_xlabel('Temperature')
ground truth:    client.send('HTTP/1.0 200 OK\r\n')
bleu socre 0.186245115152549
----------------------answer 12, this one cost 0.022728919982910156s, total c

----------------------answer 15, this one cost 0.040129899978637695s, total cost 0.9632327556610107s
my code:    concatenate(concatenate(zip(zip(b))))
ground truth:    """:""".join(str(x) for x in b)
bleu socre 0.26669676296222256
----------------------answer 16, this one cost 0.03284311294555664s, total cost 0.9970808029174805s
my code:    re.__class__('_(?:for|or|and)_', 'Current', 'â')
ground truth:    Entry.objects.filter()[:1].get()
bleu socre 0.6930977286178778
----------------------answer 17, this one cost 0.025252103805541992s, total cost 1.0235378742218018s
my code:    np.sqrt(a)
ground truth:    a.sum(axis=1)
bleu socre 0.6474591278836639
----------------------answer 18, this one cost 0.020557165145874023s, total cost 1.044785976409912s
my code:    plt.find_element_by_xpath('test.csv')
ground truth:    warnings.simplefilter('always')
bleu socre 0.5773502691896257
----------------------answer 19, this one cost 0.03206610679626465s, total cost 1.0776457786560059s
my code:    [i

...

----------------------answer 476, this one cost 0.04405093193054199s, total cost 20.910648822784424s
my code:    u = [s.split('@', '', '')]
ground truth:    [chr(i) for i in range(127)]
bleu socre 0.3381307292971254
----------------------answer 477, this one cost 0.030154943466186523s, total cost 20.94168972969055s
my code:    re.compile('\\w+(?:-\\w+)+', 'rb')
ground truth:    newFile.write(struct.pack('5B', *newFileBytes))
bleu socre 0.4041031009353247
----------------------answer 478, this one cost 0.03703904151916504s, total cost 20.980341911315918s
my code:    re.sub('^(.+)(?:\\n|\\r\\n?)((?:(?:\\n|\\r\\n?).+)+)', '', '')
ground truth:    re.sub('^[A-Z0-9]*(?![a-z])', '', string)
bleu socre 0.17001078098404226
----------------------answer 479, this one cost 0.017911195755004883s, total cost 20.999193906784058s
my code:    dict(d)
ground truth:    list(dict.keys())[-1]
bleu socre 0.1259436721879507
----------------------answer 480, this one cost 0.023858070373535156s, total cost 21

----------------------answer 484, this one cost 0.031110048294067383s, total cost 21.124616861343384s
my code:    plt.find_all("//*[contains(text(), 'My Button')]")
ground truth:    AuthorizedEmail.objects.filter(group=group).order_by('-added')[0]
bleu socre 0.4342906676853412
----------------------answer 485, this one cost 0.036955833435058594s, total cost 21.162636756896973s
my code:    re.sub('[bcdfghjklmnpqrstvwxyz]+', s)
ground truth:    re.findall('Test([0-9.]*[0-9]+)', text)
bleu socre 0.2827135566160212
----------------------answer 486, this one cost 0.029938936233520508s, total cost 21.193474769592285s
my code:    re.compile('a^', '')
ground truth:    re.findall('Test([\\d.]*\\d+)', text)
bleu socre 0.2525271432858626
----------------------answer 487, this one cost 0.019779682159423828s, total cost 21.21410369873047s
my code:    datetime.set_xticklabels('some')
ground truth:    os.system('powershell.exe', 'script.ps1')
bleu socre 0.23914346158980815
----------------------answe

----------------------answer 491, this one cost 0.03995394706726074s, total cost 21.354415893554688s
my code:    [len(x) for x in lst]
ground truth:    next(i for i, x in enumerate(lst) if not isinstance(x, bool) and x == 1)
bleu socre 0.12679444017016808
----------------------answer 492, this one cost 0.030000925064086914s, total cost 21.385339975357056s
my code:    a = [a for x in a]
ground truth:    a[:] = [(x - 13) for x in a]
bleu socre 0.24385000379404742
----------------------answer 493, this one cost 0.03403592109680176s, total cost 21.420130729675293s
my code:    f = os.Popen('file.txt')
ground truth:    random.choice(os.listdir('C:\\'))
bleu socre 0.45252910179589845
----------------------answer 494, this one cost 0.02486705780029297s, total cost 21.445799827575684s
my code:    np.concatenate(a)
ground truth:    max(x.min(), x.max(), key=abs)
bleu socre 0.11380295453101374


----------------------answer 495, this one cost 0.29209399223327637s, total cost 21.738972902297974s
my code:    """ """.(None)
ground truth:    re.findall('"(http.*?)"', s, re.MULTILINE | re.DOTALL)
bleu socre 0.14012952784975244
----------------------answer 496, this one cost 0.04464006423950195s, total cost 21.7846097946167s
my code:    re.sub('\\b(this|string)\\b', '<markup>\\1</markup>', dir)
ground truth:    re.findall('http://[^t][^s"]+\\.html', document)
bleu socre 0.45180100180492244
----------------------answer 497, this one cost 0.02951502799987793s, total cost 21.81509780883789s
my code:    re.sub(';', '', s)
ground truth:    mystring.replace(' ', '! !').split('!')
bleu socre 0.2769950866409216
----------------------answer 498, this one cost 0.021956920623779297s, total cost 21.83792281150818s
my code:    txt = os.Popen('path')
ground truth:    open(path, 'r')
bleu socre 0.4854917717073234
----------------------answer 499, this one cost 0.029181957244873047s, total cost 21.

----------------------answer 500, this one cost 0.2655179500579834s, total cost 22.134121894836426s
my code:    """ """.</primitive>(None)
ground truth:    a[:, (np.newaxis)]
bleu socre 0.6803749333171202
["os.system('taskkill /im')", "webbrowser.write('log', 'libs')", 'map(i)', 're.findall(\'filename.png\', \'"\\\\1"\', \'title\')', '"""""".join(\'true\')', "pd.DataFrame(key=lambda k=d2.keys(): 'y3')", "pd.DataFrame(key=lambda k=d2.keys(): 'y3')", 'os.system(None)', 'concatenate(concatenate(x, x))', "df.loc('type').diff('state')", "ax.set_xlabel('Temperature')", "datetime.info('test.sh')", "re.split('[', '', par)", "re.findall('dir', '.', 'this')", 'concatenate(concatenate(zip(zip(b))))', "re.__class__('_(?:for|or|and)_', 'Current', 'â')", 'np.sqrt(a)', "plt.find_element_by_xpath('test.csv')", '[int(x) for x in x]', 'os.system(\'/bin/bash -c "echo\')', '"""This""".join(\'cc\')', '[int(x, x) for x in s]', "[s.split(',', '', '')]", "plt.set_yscale('log')", "soup.findAll('{http://www.w3.

In [52]:
sum(bleu_scores)/len(bleu_scores)

0.3115922966900168

In [40]:
outfile.close()

In [42]:
code_list

["os.system('%Y-%m-%d -c')",
 "webbrowser.listdir('log', 'libs')",
 'map(lst)',
 "re.findall('filename.png', 'D:\\\\testdir', '', 'title')",
 "re.replace('')",
 'dict(list(d.values()))',
 'dict(list(d.values()))',
 "subprocess.set_title('script2.py', '$%s', 'neededArgumetGoHere', '+20')",
 "a.concatenate('b')",
 "np.einsum('ij,kj->jik', 'b')",
 "ax.set_xlabel('Temperature')",
 "plt.parse('a', '')",
 "re.findall('{http://www.w3.org/2002/07/owl#}Class', '3.1415', '')",
 "re.findall('filename.png', '*', 'this')",
 'concatenate(concatenate(zip(b)))',
 "re.findall('_(?:for|or|and)_', 'sad_pandas_and_happy_cats_for_people', 'Python'\n    )",
 'sorted(zip(arr))',
 "dparser.urldefrag('monkey')",
 'items(set(a))',
 "subprocess.system('grep -c', 'rt', 'some.file')",
 're.findall("//a[starts-with(text(),\'some", \'http://example.org/#comments\', \'\')',
 "s.import_module('11B', 'a.b')",
 ".findall('11B', '')",
 "plt.info('log')",
 "re.findall('#(\\\\w+)', '\\\\\\\\\\\\g<0>')",
 "os.system('%Y-%m-